In [61]:
!pip3 install timm
!pip install scikit-learn
import timm
import torch

### Loading Data

In [62]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((384, 384)),  # match model input
    transforms.ToTensor(),          # convert to tensor
])

dataset = datasets.ImageFolder('potato_dataset', transform=transform)
print(dataset.class_to_idx) 

{'not_potato': 0, 'potato': 1}


In [72]:
# Split into train/test
from torch.utils.data import random_split

# Define sizes
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size  # to ensure total size is preserved

# Split dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Print sizes
print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 138, Validation size: 17, Test size: 18


In [74]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

### Model Creation and Training

In [65]:
model = timm.create_model('nextvit_small.bd_ssld_6m_in1k_384', pretrained=True)
model.eval()

all_features = []
all_labels = []

for imgs, labels in train_loader:
    with torch.no_grad():
        feats = model.forward_features(imgs)  
        feats = feats.mean(dim=[2, 3])        
        all_features.append(feats)
        all_labels.append(labels)

# Stack into single tensors
X_train = torch.cat(all_features).cpu().numpy()
y_train = torch.cat(all_labels).cpu().numpy()

print(X_train.shape)
print(y_train.shape)

2025-05-03 20:39:25,473 - INFO - timm.models._builder - Loading pretrained weights from Hugging Face hub (timm/nextvit_small.bd_ssld_6m_in1k_384)
2025-05-03 20:39:25,637 - INFO - timm.models._hub - [timm/nextvit_small.bd_ssld_6m_in1k_384] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


(138, 1024)
(138,)


In [66]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1000,verbose=1,solver='saga')
logreg.fit(X_train, y_train)

Epoch 1, change: 1
Epoch 2, change: 0.18548656
Epoch 3, change: 0.07523562
Epoch 4, change: 0.084918685
Epoch 5, change: 0.04510482
Epoch 6, change: 0.043183882
Epoch 7, change: 0.031229367
Epoch 8, change: 0.029952811
Epoch 9, change: 0.027661178
Epoch 10, change: 0.020645773
Epoch 11, change: 0.018465379
Epoch 12, change: 0.01848712
Epoch 13, change: 0.014912821
Epoch 14, change: 0.014110019
Epoch 15, change: 0.013276928
Epoch 16, change: 0.011531617
Epoch 17, change: 0.010900386
Epoch 18, change: 0.010092418
Epoch 19, change: 0.0094600283
Epoch 20, change: 0.0088418331
Epoch 21, change: 0.007325002
Epoch 22, change: 0.0073754671
Epoch 23, change: 0.0068323659
Epoch 24, change: 0.0064827679
Epoch 25, change: 0.0057790894
Epoch 26, change: 0.0054550548
Epoch 27, change: 0.0050503239
Epoch 28, change: 0.0046785972
Epoch 29, change: 0.0045262296
Epoch 30, change: 0.0040744604
Epoch 31, change: 0.0041184626
Epoch 32, change: 0.0037028026
Epoch 33, change: 0.0035955166
Epoch 34, change: 0

/Users/funlau/Documents/potato/.venv/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=1000, solver='saga', verbose=1)

In [71]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log_loss',  # equivalent to logistic regression
                    max_iter=1000,
                    tol=1e-3,
                    verbose=1,
                    random_state=42)

clf.fit(X_train, y_train)

-- Epoch 1
Norm: 330.28, NNZs: 1024, Bias: -27.275848, T: 138, Avg. loss: 7.196850
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 320.66, NNZs: 1024, Bias: -21.857249, T: 276, Avg. loss: 0.633356
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 289.35, NNZs: 1024, Bias: -21.857249, T: 414, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 263.60, NNZs: 1024, Bias: -21.857249, T: 552, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 242.06, NNZs: 1024, Bias: -21.857249, T: 690, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 223.78, NNZs: 1024, Bias: -21.857249, T: 828, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 208.06, NNZs: 1024, Bias: -21.857249, T: 966, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 8
Norm: 194.41, NNZs: 1024, Bias: -21.857249, T: 1104, Avg. loss: 0.000000
Total training time: 0.01 seconds.
Convergence after 8 epochs took 0.01 seconds


SGDClassifier(loss='log_loss', random_state=42, verbose=1)

### Model Evaluation and Testing

In [ ]:
#Extract features from test images
all_test_features = []
all_test_labels = []

for imgs, labels in test_loader:
    with torch.no_grad():
        feats = model.forward_features(imgs)  
        feats = feats.mean(dim=[2, 3])   
             
        all_test_features.append(feats)
        all_test_labels.append(labels)

# Stack into arrays
X_test = torch.cat(all_test_features).cpu().numpy()
y_test = torch.cat(all_test_labels).cpu().numpy()
print(X_test.shape)

(35, 1024)


In [47]:

y_pred = logreg.predict(X_test)
print(y_pred)

[1 0 0 0 0 1 1 0 0 0 0 1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 1 1 1]


In [68]:
clf_pred = clf.predict(X_test)
print(clf_pred)

[1 0 0 0 0 1 1 0 0 0 0 1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 1 1 1]


In [48]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc * 100:.2f}%")

Test Accuracy: 100.00%


In [ ]:
!pip3 install matplotlib

In [ ]:
#matching the test images with their predictions
import matplotlib.pyplot as plt

class_names = dataset.classes  # ['not_potato', 'potato']

start = 0
for imgs, labels in test_loader:
    batch_size = imgs.size(0)
    preds = y_pred[start:start+batch_size]
    start += batch_size

    for i in range(batch_size):
        img = imgs[i].permute(1, 2, 0).cpu().numpy()
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Predicted: {class_names[preds[i]]}")
        plt.show()


### Save Model

In [52]:
import joblib

# Save
joblib.dump(logreg, 'potato_logreg.pkl')

['potato_logreg.pkl']